# Assignment #1: Building an inverted index

Author: Pierre Nugues


## Objectives


The objectives of this assignment are to:

- Write a program that collects all the words from a set of documents
- Build an index from the words
- Represent a document using the Tf.Idf values
- Write a short report of 1 to 2 pages on the assignment
- Read a description of an industrial system and answer a question on it
- Discuss the potential of corpora in applications


## Submission


When you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.


## Description of the assignment


### Outline


In this lab, you will build an indexer to index all the words in a corpus. Conceptually, an index consists of rows with one word per row and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.

<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>


#### Imports


Some imports. Add others as needed


In [247]:
import bz2
import math
import os
import pickle
import regex as re
import requests
import sys
import time
from zipfile import ZipFile

## Corpus


You will create an index for a corpus of Selma Lagerlöf's works: To gather the corpus, you can alternatively:

1. Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">Selma Lagerlöf</a>. The text of these novels was extracted from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
2. Or run this cell that will download the corpus and place it in your folder.


In [248]:
# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname) 
    for fname in 
    [
        "bannlyst.txt", 
        "gosta.txt", 
        "herrgard.txt", 
        "jerusalem.txt", 
        "kejsaren.txt", 
        "marbacka.txt", 
        "nils.txt", 
        "osynliga.txt", 
        "troll.txt"
    ]
]

def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""
    
    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()
        
        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith("__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)
            
        print("Done!")
        
# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")
    
SELMA_FILES

Selma has been downloaded.


['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

### Running the indexer (optional)


In a production context, your final program would take a corpus as input (here the Selma Lagerlöf's novels) and create an index of all the words with their positions. You should be able to run it this way:

<pre>$ python indexer.py folder_name</pre>

In this lab, you will write the index in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.


## Programming the Indexer


To make programming easier, you will split this exercise into five steps:

1. Index one file;
2. Read the content of a folder
3. Create a master index for all the files
4. Use tfidf to represent the documents (novels)
5. Compare the documents of a collection

You will use dictionaries to represent the postings.


### Indexing one file


#### Description


<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use the <tt>finditer()</tt> method to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>


Below is an excerpt of the index of the `bannlyst.txt` text for the words <i>gjord</i>, <i>uppklarnande</i>, and <i>stjärnor</i>. The data is stored in a dictionary:

<pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
</pre>

where the word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at position 8617, and <i>stjärnor</i>, once at position 8641.


#### Writing a tokenizer


Write a **Unicode** regular expression to find words defined as sequences of letters. Using the Unicode notation is compulsory. If you are not sure, please refer to the lecture.


In [249]:
# Write your regex here
regex = "\p{L}+"

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
/var/folders/b2/dwmc73n52m15hgf8m1f1h5v80000gn/T/ipykernel_71328/3341073934.py:2: SyntaxWarning: invalid escape sequence '\p'
  regex = "\p{L}+"


In [250]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write `tokenize(text)` function to tokenize a text. Return their positions.


In [251]:
# Write your code here
def tokenize(text):
  xs = re.finditer(regex, text)

  return xs

In [252]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

[<regex.Match object; span=(0, 2), match='En'>,
 <regex.Match object; span=(3, 7), match='gång'>,
 <regex.Match object; span=(8, 12), match='hade'>,
 <regex.Match object; span=(13, 15), match='de'>,
 <regex.Match object; span=(16, 18), match='på'>,
 <regex.Match object; span=(19, 27), match='Mårbacka'>,
 <regex.Match object; span=(28, 30), match='en'>,
 <regex.Match object; span=(31, 39), match='barnpiga'>,
 <regex.Match object; span=(41, 44), match='som'>,
 <regex.Match object; span=(45, 50), match='hette'>,
 <regex.Match object; span=(51, 55), match='Back'>,
 <regex.Match object; span=(56, 61), match='Kajsa'>]

#### Extracting indices


Write a `text_to_idx(words)` function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the values a list of positions.


In [253]:
# Write your code here
def text_to_idx(words):
  dic = {}

  for word in words:
    if word.group() in dic:
      dic[word.group()].append(word.start())
    else:
      dic[word.group()] = [word.start()]

  return dic


In [254]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 28],
 'gång': [3],
 'hade': [8],
 'de': [13],
 'på': [16],
 'mårbacka': [19],
 'barnpiga': [31],
 'som': [41],
 'hette': [45],
 'back': [51],
 'kajsa': [56]}

#### Reading one file


Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`


In [255]:
# Write your code here

idx = text_to_idx(tokenize(open("Selma/marbacka.txt").read().lower()))



In [256]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

#### Saving the index


Save your index in a file so that you can reuse it. Use the pickle module.


In [257]:
# Write your code here
index_file = 'marbacka.idx'

db_file = open(index_file, 'ab')

pickle.dump(idx, db_file)
db_file.close()


Read back your file and store the content in `idx`


In [258]:
# Write your code here
db_file = open(index_file, 'rb')
idx = pickle.load(db_file)
db_file.close()

In [259]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

### Reading the content of a folder


Write a `get_files(dir, suffix)` function that reads all the files in a folder with a specific `suffix` (txt). You will need the Python `os` package, see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You will return the file names in a list.


You can reuse this function:


In [260]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [261]:
# Write your code here
get_files("./Selma", ".txt")

['troll.txt',
 'kejsaren.txt',
 'marbacka.txt',
 'herrgard.txt',
 'nils.txt',
 'osynliga.txt',
 'jerusalem.txt',
 'bannlyst.txt',
 'gosta.txt']

### Creating a master index


Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions: a posting list
Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:


In [262]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
313784, 409998, and 538165.


In [263]:
# write your code here
master_index = {}

def make_idx(book):
    dic = text_to_idx(tokenize(open("./Selma/"+book).read().lower()))
    
    for word in dic:
        if word in master_index:
            master_index[word].update({book: dic[word]})
        else:
            master_index[word] = {book: dic[word]}

corpus_files = get_files("./Selma/", ".txt")
for file in corpus_files:
    make_idx(file)

In [264]:
master_index['samlar']

{'troll.txt': [641880, 654233],
 'nils.txt': [51805, 118943],
 'osynliga.txt': [399121],
 'gosta.txt': [313784, 409998, 538165]}

In [265]:
master_index['mårbacka']

{'troll.txt': [226291, 387634, 392959],
 'marbacka.txt': [16,
  139,
  752,
  1700,
  2582,
  3324,
  15117,
  15404,
  27794,
  42175,
  49126,
  50407,
  52053,
  60144,
  63374,
  64910,
  67182,
  67330,
  67799,
  67824,
  69232,
  71328,
  72099,
  74147,
  74255,
  74614,
  76610,
  76884,
  77138,
  77509,
  77787,
  77936,
  78574,
  80597,
  81782,
  82003,
  84363,
  84786,
  85251,
  89837,
  97093,
  98642,
  100474,
  105063,
  105298,
  105721,
  108710,
  109133,
  112844,
  113725,
  114997,
  115583,
  115833,
  116368,
  116557,
  121896,
  124823,
  126409,
  126542,
  128758,
  130976,
  131939,
  132826,
  136914,
  137187,
  137872,
  139196,
  140721,
  142324,
  146781,
  151497,
  154335,
  155139,
  155438,
  155886,
  156405,
  158108,
  159817,
  160107,
  161158,
  162085,
  165847,
  168316,
  168528,
  169111,
  170333,
  172684,
  182047,
  182427,
  186362,
  189535,
  190999,
  191110,
  193177,
  196686,
  202552,
  206340,
  207789,
  208382,
  2098

Save your master index in a file and read it again


In [266]:
# Write your code here
index_file = 'masterIndx.idx'

db_file = open(index_file, 'ab')

pickle.dump(master_index, db_file)
db_file.close()

db_file = open(index_file, 'rb')
master_index = pickle.load(db_file)
db_file.close()




In [267]:
master_index['samlar']

{'troll.txt': [641880, 654233],
 'nils.txt': [51805, 118943],
 'osynliga.txt': [399121],
 'gosta.txt': [313784, 409998, 538165]}

#### Concordances


Write a `concordance(word, master_index, window)` function to extract the concordances of a `word` within a window of `window` characters


In [268]:
# Write your code here
def concordance(word, master_index, window):
    if word not in master_index:
        return
    
    occurences = master_index[word]

    for occurence in occurences:
        book = open(f"./Selma/{occurence}").read().lower().replace("\n", " ")

        print(occurence)
        for index in occurences[occurence]:
            print(f"\t{book[index-window:index+window]}")

In [269]:
concordance('samlar', master_index, 25)

troll.txt
	en örtkunnig läkare, som samlar in markens växter 
	älper dem, och medan hon samlar och handlar för de
nils.txt
	 bara, att du i all hast samlar ihop så mycket bos
	ar stannat hemma, och nu samlar de sig för att int
osynliga.txt
	 till höger i kärran och samlar just ihop tömmarna
gosta.txt
	om ligger nära borg, och samlar ihop ett litet mid
	lika förstämda.  men hon samlar upp allt detta som
	n ensam i livet.  därmed samlar han korten tillhop


### Representing Documents with tf-idf


Once you have created the index, you will represent each document in your corpus as a dictionary. The keys of these dictionaries will be the words and you will define the value of a word with the tf-idf metric:

1. Read the description of the tf-idf measure on Wikipedia (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>)
2. After reading the description, you probably realized that there are multiple definitions of tf-idf. In this assignment,

- Tf will be the relative frequency of the term in the document and
- idf, the logarithm base 10 of the inverse document frequency.

You have below the tf-idf values for a few words. In our example, the word <i>gås</i> has the value 0 in bannlyst.txt and the value 0.000101001964 in nils.txt

<pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
</pre>


Conceptually, the tf-idf representation is a vector. In your program, you will keep this idea and use all the words in the corpus as keys: Each dictionary will include all the words of the corpus as keys. The value of the key is then possibly 0, meaning that the word in not in the document or is in all the documents as for the word `nils` in `gosta.tx`.

As further work, you may think of optimizing this part.


In [270]:
# Write your code here
tfidf = {}

all_words = []
is_in_n_books = {}
nbr_of_words_in_book = {}
nbr_files = len(corpus_files)

def read_words(filename):
    result = "" 
    for line in open(f"./Selma/{filename}", encoding='utf-8'):
        line.lower().strip()
        result += line.lower()
    return result
  
for file in corpus_files:
  book = read_words(file)
  tokens = tokenize(book)
  words = text_to_idx(tokens)
  tfidf[file] = {}

  nbr_words = 0
  for word in words:
    nbr_words += len(words[word])

  nbr_of_words_in_book[file] = nbr_words

  for word in words.keys():
    all_words.append(word)
    is_in_n_books[word] = is_in_n_books.get(word, 0) + 1

for word in set(all_words):
  idf = math.log10(nbr_files / is_in_n_books[word])

  for file in corpus_files:
    occurences = len(master_index.get(word, {}).get(file, []))
    count = nbr_of_words_in_book[file]
    tf = occurences / count

    tfidf[file][word] = tf * idf



In [271]:
tfidf['troll.txt']['känna']

0.0

In [272]:
tfidf['troll.txt']['nils']

2.1481617488686316e-06

### Comparing Documents


Using the cosine similarity, compare all the pairs of documents with their tf-idf representation and present your results in a table. You will include this table in your report.


#### Cosine similarity


Write a function computing the cosine similarity between two documents: `cosine_similarity(document1, document2)`


In [273]:
# Write your code here
def cosine_similarity(document1, document2):
    values1 = tfidf[document1]
    values2 = tfidf[document2]
    res = 0
    len1 = 0
    len2 = 0
    for x in (values1.keys() | values2.keys()):
        value1 = 0
        if x in values1:
            value1 = values1[x]
        
        value2 = 0
        if x in values2:
            value2 = values2[x]

        res += value1 * value2 

        len1 += value1 * value1
        len2 += value2 * value2

    return res / ((math.sqrt(len1)*math.sqrt(len2)))

#### Similarity matrix


Compute the similarity matrix between the documents of the corpus. While computing the similarities, you will record the two most similar documents that you will call `most_sim_doc1` and `most_sim_doc2`.


In [274]:
print(corpus_files)

most_sim_doc1 = ""
most_sim_doc2 = ""
max_similarity = 0

for f1 in corpus_files:
  scores = []
  for f2 in corpus_files:
    sim = cosine_similarity(f1, f2)
    scores.append(sim)

    if sim > max_similarity and f1 != f2:
      max_similarity = sim
      most_sim_doc1 = f1
      most_sim_doc2 = f2

  formatted_scores = " ".join(f"{s:8.4f}" for s in scores)
  print(f"{f1:<20} {formatted_scores}")


['troll.txt', 'kejsaren.txt', 'marbacka.txt', 'herrgard.txt', 'nils.txt', 'osynliga.txt', 'jerusalem.txt', 'bannlyst.txt', 'gosta.txt']
troll.txt              1.0000   0.1802   0.1487   0.0041   0.1905   0.1927   0.0071   0.0897   0.1961
kejsaren.txt           0.1802   1.0000   0.0708   0.0007   0.0503   0.0513   0.0018   0.0243   0.0483
marbacka.txt           0.1487   0.0708   1.0000   0.0036   0.0865   0.0940   0.0048   0.0377   0.0811
herrgard.txt           0.0041   0.0007   0.0036   1.0000   0.0051   0.0048   0.3709   0.0009   0.0031
nils.txt               0.1905   0.0503   0.0865   0.0051   1.0000   0.1114   0.0046   0.0520   0.1059
osynliga.txt           0.1927   0.0513   0.0940   0.0048   0.1114   1.0000   0.0283   0.0525   0.1250
jerusalem.txt          0.0071   0.0018   0.0048   0.3709   0.0046   0.0283   1.0000   0.0064   0.0043
bannlyst.txt           0.0897   0.0243   0.0377   0.0009   0.0520   0.0525   0.0064   1.0000   0.0495
gosta.txt              0.1961   0.0483   0.0811 

Give the name of the two novels that are the most similar.


In [275]:
print("Most similar:", most_sim_doc1, most_sim_doc2, "Similarity:", max_similarity)

Most similar: herrgard.txt jerusalem.txt Similarity: 0.3709202232020964


<h2>Turning in your assigment</h2>


Now your are done with the program. To complete this assignment, you will:

1. Write a short individual report on your program,
2. Read the text <i>Challenges in Building Large-Scale Information Retrieval Systems</i> about the history of <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a> by <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a> and write a short comment on it. See the details below.
3. Watch the speech delivered by Steve Jobs in 1985 (https://www.youtube.com/watch?v=NT6oUsn3ANA) and describe superficially in one paragraph how computers can amass knowledge and have dialogues with us.

You will submit your report as well as your notebook (for archiving purposes) to Canvas: <https://canvas.education.lu.se/>. You have two separate places: one of the report and the other for the program.

I suggest that you use this structure for your report:

1. Objectives and dataset
2. Method and program structure, where you should outline your program and possibly describe difficult parts.
3. Results
4. Conclusion
5. Answer to possible questions

In your report of about two pages:

1. You will describe your indexer (Sect. Method) and comment your results (Sect. Result); in this description, you will write the regular expression you used for the tokenization (Sect. Method) and include the similarity matrix (Sect. Results);
2. In Jeff Dean's document, slide 45 describes an indexing technique. Tell how your index encoding is related to what Google did. (Sect. Other questions).
3. Referring to Jobs's speech, comment shortly how computers can amass knowledge and have dialogues with us.

To write your report, use Latex. This is mandatory as it will help you structure your text. You will then upload a PDF file in Canvas. You can try Overleaf (<a href="https://www.overleaf.com/">www.overleaf.com</a>) as it makes it easy to create Latex document.

The submission deadline is September 19, 2025. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.
